In [1]:
from ultralytics import YOLO
import cv2
import pandas as pd

In [2]:
model = YOLO('yolov8m-pose.pt')  # 경량 포즈 모델

results = model.predict(source='bad_posture_20241012_174220_png.rf.9ddba68fb2504d13324bdff2612f9126.jpg', save=False, conf=0.5)

# keypoint 추출
keypoints = results[0].keypoints.xy[0].cpu().numpy()  # (17, 2)
print(keypoints)


image 1/1 c:\VisualStudio-WorkSpace\conetComputerVision\personalproject\bad_posture_20241012_174220_png.rf.9ddba68fb2504d13324bdff2612f9126.jpg: 640x640 1 person, 381.9ms
Speed: 2.5ms preprocess, 381.9ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
[[     341.03      177.81]
 [          0           0]
 [     332.53      155.67]
 [          0           0]
 [     273.52      144.86]
 [     246.36      236.85]
 [     198.67      238.37]
 [     287.35      351.22]
 [     208.83      384.06]
 [     408.31      333.34]
 [     416.21      357.45]
 [     263.48      447.98]
 [     235.32       477.5]
 [     459.86      406.27]
 [     468.31      439.81]
 [          0           0]
 [          0           0]]


In [2]:
import torch

print("✅ CUDA 사용 가능:", torch.cuda.is_available())
print("🖥️ GPU 이름:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")
print("💡 PyTorch 버전:", torch.__version__)


✅ CUDA 사용 가능: True
🖥️ GPU 이름: NVIDIA GeForce RTX 3070 Ti Laptop GPU
💡 PyTorch 버전: 2.4.1+cu121


In [27]:
from ultralytics import YOLO
import cv2

models = {
    'nano': YOLO("yolo_weights/yolov8n-pose.pt"),
    'small': YOLO("yolo_weights/yolov8s-pose.pt"),
    'medium': YOLO("yolo_weights/yolov8m-pose.pt")
}

image = cv2.imread("roboflow04/train/images/-_-_-_jpg.rf.367c05f849499d73d37fd2db5f14bf3b.jpg")

for name, model in models.items():
    results = model(image)
    keypoints = results[0].keypoints
    count = len(keypoints.data) if keypoints is not None else 0
    print(f"[{name.upper()}] 감지된 사람 수: {count}")



0: 640x640 1 person, 28.3ms
Speed: 6.0ms preprocess, 28.3ms inference, 5.1ms postprocess per image at shape (1, 3, 640, 640)
[NANO] 감지된 사람 수: 1

0: 640x640 1 person, 61.3ms
Speed: 4.1ms preprocess, 61.3ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)
[SMALL] 감지된 사람 수: 1

0: 640x640 1 person, 75.3ms
Speed: 4.3ms preprocess, 75.3ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)
[MEDIUM] 감지된 사람 수: 1


In [26]:
import os
import cv2
import torch
import numpy as np
from ultralytics import YOLO

# === 설정 ===
MODEL_PATH = 'yolo_weights/yolov8m-pose.pt'
IMAGE_PATH = 'roboflow02/train/images/image_20241009_224235_jpg.rf.a3000c49ad1e06daab36ff2daaccea2e.jpg'
IMAGE_PATH = 'roboflow04/train/images/-_-_-_jpg.rf.367c05f849499d73d37fd2db5f14bf3b.jpg'

# === 모델 로드
model = YOLO(MODEL_PATH)

# === 이미지 로드 및 추론
image = cv2.imread(IMAGE_PATH)
results = model(image)
keypoints = results[0].keypoints

if keypoints is None or len(keypoints.data) == 0:
    print("❌ 사람 감지 실패")
else:
    image_copy = image.copy()
    num_people = keypoints.data.shape[0]
    print(f"👥 감지된 사람 수: {num_people}")

    # 색상 팔레트 (사람별 색 다르게)
    colors = [(0,255,0), (255,0,0), (0,0,255), (255,255,0), (255,0,255)]

    for i in range(num_people):
        person_kpts = keypoints.data[i][:, :2].cpu().numpy()  # (17, 2)
        color = colors[i % len(colors)]

        for (x, y) in person_kpts:
            if x > 0 and y > 0:  # 유효 keypoint만
                cv2.circle(image_copy, (int(x), int(y)), radius=4, color=color, thickness=-1)

    # 결과 보기
    cv2.imshow('All Keypoints (Multi-Person)', image_copy)
    cv2.waitKey(0)
    cv2.destroyAllWindows()



0: 640x640 1 person, 65.2ms
Speed: 4.4ms preprocess, 65.2ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)
👥 감지된 사람 수: 1


In [ ]:
import os
import cv2
import pandas as pd
from ultralytics import YOLO

# === 설정 ===
SPLITS = ['train', 'valid', 'test']
BASE_DIR = 'roboflow04'
OUTPUT_DIR = 'MediumKeypointOutput/04'
MODEL_PATH = 'yolo_weights/yolov8m-pose.pt'

CLASS_MAP = {
    0: 'bad',
    1: 'good'
}

model = YOLO(MODEL_PATH)

def get_label_from_txt(txt_path):
    if not os.path.exists(txt_path):
        return None
    with open(txt_path, 'r') as f:
        lines = f.readlines()
        if not lines:
            return None
        class_idx = int(lines[0].split()[0])
        return CLASS_MAP.get(class_idx)

# === 각 split별로 처리 ===
for split in SPLITS:
    image_dir = os.path.join(BASE_DIR, split, 'images')
    label_dir = os.path.join(BASE_DIR, split, 'labels')
    output_csv = os.path.join(OUTPUT_DIR, f'{split}_keypoints_medium04.csv')

    rows = []

    for file_name in os.listdir(image_dir):
        if not file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        txt_name = file_name.rsplit('.', 1)[0] + '.txt'
        label_path = os.path.join(label_dir, txt_name)
        label = get_label_from_txt(label_path)

        if label is None:
            print(f'[❌] 라벨 없음 또는 라벨 파일 문제: {file_name}')
            continue

        image_path = os.path.join(image_dir, file_name)
        image = cv2.imread(image_path)

        results = model(image)
        keypoints = results[0].keypoints

        # 감지된 사람 수 확인
        if keypoints is not None:
            num_persons = len(keypoints.xy)
            print(f'[👤] 감지된 사람 수: {num_persons} - 파일명: {file_name}')
        else:
            print(f'[❌] Keypoint 없음: {file_name}')
            continue


        kp = keypoints.data[0][:, :2].cpu().numpy().flatten()
        row = list(kp) + [label]
        rows.append(row)

    if not rows:
        print(f'[⚠️] {split} split에서 유효한 데이터 없음.')
        continue

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    columns = [f'x{i}' if i % 2 == 0 else f'y{i//2}' for i in range(34)] + ['label']
    df = pd.DataFrame(rows, columns=columns)
    df.to_csv(output_csv, index=False)
    print(f'[✅] {split} → {output_csv} 저장 완료.')
    


[❌] 라벨 없음 또는 라벨 파일 문제: -1-_JPG_jpg.rf.190ebc51ac0c589528ec269ddee22727.jpg

0: 640x640 1 person, 14.7ms
Speed: 2.6ms preprocess, 14.7ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)
[👤] 감지된 사람 수: 1 - 파일명: -1-_JPG_jpg.rf.5b9b069da6435e2542e8b7fd2358504a.jpg

0: 640x640 1 person, 14.8ms
Speed: 2.5ms preprocess, 14.8ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)
[👤] 감지된 사람 수: 1 - 파일명: -11-_JPG_jpg.rf.c63f7fef285caeb719d5043a61023650.jpg

0: 640x640 1 person, 14.6ms
Speed: 1.7ms preprocess, 14.6ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)
[👤] 감지된 사람 수: 1 - 파일명: -4-_JPG_jpg.rf.a946055b0ef250b38b08811dbed608aa.jpg
[❌] 라벨 없음 또는 라벨 파일 문제: -6-_JPG_jpg.rf.20b3da9a693303bdf0af4ddac7cd286e.jpg

0: 640x640 1 person, 14.7ms
Speed: 1.8ms preprocess, 14.7ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)
[👤] 감지된 사람 수: 1 - 파일명: -63-_jpg.rf.b551f0850b38007cf5d41939f10af208.jpg

0: 640x640 1 person, 14.7ms
Speed: 1.9ms prepro

In [19]:
import os
import cv2
import pandas as pd
import torch
from ultralytics import YOLO

# === 설정 ===
SPLITS = ['train', 'valid', 'test']
BASE_DIR = 'roboflow04'
OUTPUT_DIR = 'MediumKeypointOutput/04'
MODEL_PATH = 'yolo_weights/yolov8m-pose.pt'

CLASS_MAP = {
    0: 'bad',
    1: 'good'
}

model = YOLO(MODEL_PATH)

# 숫자 그대로 반환
def get_label_from_txt(txt_path):
    if not os.path.exists(txt_path):
        return None
    with open(txt_path, 'r') as f:
        lines = f.readlines()
        if not lines:
            return None
        class_idx = int(lines[0].split()[0])
        return class_idx  # 문자열이 아니라 숫자 반환!


# === 각 split별로 처리 ===
for split in SPLITS:
    image_dir = os.path.join(BASE_DIR, split, 'images')
    label_dir = os.path.join(BASE_DIR, split, 'labels')
    output_csv = os.path.join(OUTPUT_DIR, f'{split}_keypoints_medium04.csv')

    rows = []

    for file_name in os.listdir(image_dir):
        if not file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        txt_name = file_name.rsplit('.', 1)[0] + '.txt'
        label_path = os.path.join(label_dir, txt_name)
        label = get_label_from_txt(label_path)

        if label is None:
            print(f'[❌] 라벨 없음 또는 라벨 파일 문제: {file_name}')
            continue

        image_path = os.path.join(image_dir, file_name)
        image = cv2.imread(image_path)

        results = model(image)
        keypoints = results[0].keypoints

        if keypoints is None or len(keypoints.data) == 0:
            print("❌ 사람 없음")
            continue

        num_people = len(keypoints.data)

        # 1명일 때는 그냥 0번 사용
        if num_people == 1:
            center_idx = 0
        else:
            # 여러 명이면 중심에 가까운 사람 고르기
            image_center_x = image.shape[1] // 2
            nose_x = keypoints.xy[:, 0]  # 사람들의 nose x좌표
            center_idx = torch.argmin(torch.abs(nose_x - image_center_x)).item()

        # 혹시 계산된 인덱스가 너무 크면 건너뛰기
        if center_idx >= num_people:
            print("❗ 잘못된 사람 번호. 건너뜀")
            continue

        # 선택된 사람의 keypoint만 저장
        kp = keypoints.data[center_idx][:, :2].cpu().numpy().flatten()
        row = list(kp) + [label]
        rows.append(row)


    if not rows:
        print(f'[⚠️] {split} split에서 유효한 데이터 없음.')
        continue

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    columns = [f'x{i}' if i % 2 == 0 else f'y{i//2}' for i in range(34)] + ['label']
    df = pd.DataFrame(rows, columns=columns)
    df.to_csv(output_csv, index=False)
    print(f'[✅] {split} → {output_csv} 저장 완료.')


[❌] 라벨 없음 또는 라벨 파일 문제: -1-_JPG_jpg.rf.190ebc51ac0c589528ec269ddee22727.jpg

0: 640x640 1 person, 14.5ms
Speed: 2.8ms preprocess, 14.5ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 14.8ms
Speed: 2.5ms preprocess, 14.8ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 14.6ms
Speed: 2.2ms preprocess, 14.6ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)
[❌] 라벨 없음 또는 라벨 파일 문제: -6-_JPG_jpg.rf.20b3da9a693303bdf0af4ddac7cd286e.jpg

0: 640x640 1 person, 14.5ms
Speed: 1.7ms preprocess, 14.5ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 14.6ms
Speed: 2.0ms preprocess, 14.6ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)
[❌] 라벨 없음 또는 라벨 파일 문제: -7-_JPG_jpg.rf.99ae399f205cb2cae89e9fecc260a512.jpg
[❌] 라벨 없음 또는 라벨 파일 문제: -8-_JPG_jpg.rf.db186c20cd7b9ebe9b6609d47da56b12.jpg

0: 640x640 1 person, 14.6ms
Speed: 1.9ms preprocess, 14.6ms inference

In [15]:
import os
import pandas as pd
# === 설정 ===
input_path = "MediumKeypointOutput/01/test_keypoints_medium01.csv"  # 원본 CSV들이 있는 폴더
# OUTPUT_DIR = "MediumKeypointOutput/04"             # 정제된 파일을 저장할 폴더
# os.makedirs(OUTPUT_DIR, exist_ok=True)
df = pd.read_csv(input_path)
df

,x0,y0,x2,y1,x4,y2,x6,y3,x8,y4,...,y12,x26,y13,x28,y14,x30,y15,x32,y16,label
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,134.427658,0.000000,...,342.591309,270.278564,387.306427,282.569702,386.691589,243.026215,588.666687,252.773926,606.335571,good
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,353.793945,266.398285,396.411926,291.642670,385.721771,162.954895,580.651489,171.977692,591.364380,good
2,431.5339,257.796448,0.000000,0.000000,419.799957,237.884384,0.000000,0.000000,371.330505,235.238953,...,602.196838,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,good
3,good,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,bad,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,bad,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,bad,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,bad,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,255.74495,241.959412,261.928864,221.739212,0.000000,0.000000,320.800232,207.166595,0.000000,0.000000,...,473.282806,290.072906,525.924805,281.606384,496.162842,0.000000,0.000000,0.000000,0.000000,bad
9,434.44342,128.723572,0.000000,0.000000,436.739655,104.344925,0.000000,0.000000,396.347321,56.939758,...,390.051239,0.000000,0.000000,240.964859,550.567688,0.000000,0.000000,0.000000,0.000000,bad


In [20]:
import os
import pandas as pd

def parse_label_file(filepath, filename):
    rows = []
    with open(filepath, 'r') as f:
        for line in f:
            parts = line.strip().split()
            class_id = int(parts[0])
            bbox = list(map(float, parts[1:5]))
            keypoints = list(map(float, parts[5:]))  # 17 keypoints × 3 (x,y,v)
            row = [filename, class_id] + bbox + keypoints
            rows.append(row)
    return rows

def process_folder(split_name, base_path="roboflowfin"):
    label_dir = os.path.join(base_path, split_name, 'labels')
    image_dir = os.path.join(base_path, split_name, 'images')
    all_data = []

    for label_file in os.listdir(label_dir):
        if not label_file.endswith('.txt'):
            continue
        filepath = os.path.join(label_dir, label_file)
        image_filename = label_file.replace('.txt', '.jpg')  # or .png if needed
        rows = parse_label_file(filepath, image_filename)
        all_data.extend(rows)

    # 열 이름 만들기
    num_kps = (len(all_data[0]) - 6) // 3
    columns = ['filename', 'class_id', 'x_center', 'y_center', 'width', 'height']
    for i in range(num_kps):
        columns += [f'kp{i}_x', f'kp{i}_y', f'kp{i}_v']

    df = pd.DataFrame(all_data, columns=columns)
    df.to_csv(f"{split_name}_pose.csv", index=False)
    print(f"{split_name}_pose.csv 저장 완료")

# 세 개 세트 모두 처리
for split in ['train', 'valid', 'test']:
    process_folder(split)


train_pose.csv 저장 완료
valid_pose.csv 저장 완료
test_pose.csv 저장 완료
